In [1]:
from readlif.reader import LifFile
import os
import numpy as np
from tifffile import imwrite
import napari

### Patways

In [2]:
file_path  = r'R:\Aidan\ULA Timecourse_ARC _004_merged.lif'

dir_save = r'R:\Aidan\240910_tiff_movies'

### Look inside the file

In [3]:
lif_file = LifFile(file_path)

im_list = [(ind,x['name'],x['dims'], x['channels']) for ind,x in enumerate(lif_file.image_list)]
im_list

[(0, 'B3 Region1_Merged', Dims(x=9449, y=10290, z=9, t=11, m=1), 2),
 (1, 'B4 Region1_Merged', Dims(x=9726, y=9483, z=9, t=11, m=1), 2),
 (2, 'C2 Region1_Merged', Dims(x=9888, y=9586, z=9, t=11, m=1), 2),
 (3, 'C3 Region1_Merged', Dims(x=9322, y=10002, z=9, t=11, m=1), 2),
 (4, 'C4 Region1_Merged', Dims(x=9632, y=9779, z=9, t=11, m=1), 2),
 (5, 'D2 Region1_Merged', Dims(x=9760, y=9960, z=9, t=11, m=1), 2),
 (6, 'D3 Region1_Merged', Dims(x=9853, y=9863, z=9, t=11, m=1), 2),
 (7, 'D4 Region1_Merged', Dims(x=9957, y=9829, z=9, t=11, m=1), 2),
 (8,
  'B2 Region1_Merged_Processed001',
  Dims(x=9614, y=9820, z=1, t=11, m=1),
  2),
 (9,
  'B4 Region1_Merged_Processed001',
  Dims(x=9726, y=9483, z=1, t=11, m=1),
  2),
 (10,
  'C2 Region1_Merged_Processed001',
  Dims(x=9888, y=9586, z=1, t=11, m=1),
  2),
 (11,
  'C3 Region1_Merged_Processed001',
  Dims(x=9322, y=10002, z=1, t=11, m=1),
  2),
 (12,
  'C4 Region1_Merged_Processed001',
  Dims(x=9632, y=9779, z=1, t=11, m=1),
  2),
 (13,
  'D3 Reg

### Specify files to process

In [4]:
#sel_ind = range(len(im_list)) # for all available images
sel_ind = [0] # for selected images

im_list_sel = [lif_file.get_image(ind) for ind in sel_ind]
im_list_sel

['LifImage object with dimensions: Dims(x=9449, y=10290, z=9, t=11, m=1)']

### Process images

In [6]:
for im in im_list_sel:

    print(f'Processing image: {im.name}')

    for ch in range(im.channels):

        print(f'Processing channel: {ch}')

        # empty list to collect frames
        frame_list = []

        for t in range (im.dims.t):

            print(f'Processing frame: {t}')

            frame_generator = im.get_iter_z(t=t, c=ch)

            frame_stack = np.zeros((im.dims.z,im.dims.y,im.dims.x),dtype=np.uint16)
            z = 0
            for frame in frame_generator:
                
                frame_stack[z,:,:] = np.array(frame)
                z += 1

            frame_projection = np.max(frame_stack,axis=0)

            frame_list.append(frame_projection)

        # save the movie
        save_path = os.path.join(dir_save,im.name + '_ch_' + str(ch).zfill(2) + '.tif')

        imwrite(save_path, np.array(frame_list).astype('uint16'), imagej=True)